In [1]:
# Add the parent directory to the system path for proper module imports
import sys
sys.path.append("..")

In [2]:
# Import the function from the fetch_data.py file
from stock_data.fetch_data import fetch_stock_data

In [8]:
# Define tickers and start date
tickers = ["AAPL", "MSFT", "NVDA", "AMZN", "SPOT"]
start_date = "2021-01-01"

# Fetch the data
raw_data = fetch_stock_data(tickers, start_date=start_date)

# Display the fetched data
raw_data.head()

Price        Date       Close                                                 \
Ticker                   AAPL        AMZN        MSFT       NVDA        SPOT   
0      2021-01-04  125.974480  159.331497  208.882172  13.077454  311.000000   
1      2021-01-05  127.531944  160.925507  209.083755  13.367904  313.720001   
2      2021-01-06  123.239059  156.919006  203.662292  12.579825  314.239990   
3      2021-01-07  127.444374  158.108002  209.457901  13.307319  331.359985   
4      2021-01-08  128.544388  159.134995  210.734131  13.240257  353.109985   

Price         High                                     ...        Open  \
Ticker        AAPL        AMZN        MSFT       NVDA  ...        AAPL   
0       130.062977  163.600006  213.977324  13.614973  ...  129.975370   
1       128.242575  161.169006  209.678680  13.406547  ...  125.468231   
2       127.570927  159.875504  207.730746  13.208592  ...  124.329329   
3       128.135532  160.427002  210.465421  13.340976  ...  124.952339   
4       129.108992  159.531998  211.655296  13.384111  ...  128.914290   

Price                                                     Volume            \
Ticker        AMZN        MSFT       NVDA        SPOT       AAPL      AMZN   
0       163.500000  213.526340  13.068229  317.420013  143301900  88228000   
1       158.300507  208.469650  13.063742  309.480011   97664900  53110000   
2       157.324005  203.585528  13.186154  308.160004  155088000  87896000   
3       157.850006  205.379858  12.931854  317.440002  109578200  70290000   
4       159.000000  209.832161  13.325772  335.000000  105158200  70754000   

Price                                 
Ticker      MSFT       NVDA     SPOT  
0       37130100  560640000   945700  
1       23823000  322760000   610400  
2       35930700  580424000  1024800  
3       27694500  461480000   977200  
4       22956200  292528000  2199700  

[5 rows x 26 columns]